In [2]:
import tensorflow as tf
from icecream import ic
from threading import Thread
import numpy as np
from Bio import pairwise2
import time

Here I am tring to make two tensors shift to make the alignment. I need max(len) - 1 iteration which can be converted in n/2, because I can shift both extremities. However the issue now is that the two tensor are not the same iteration.

I want as results a tensor long as the number of total possiple comparison between the two sequences. The value will be representative as the sum of the differences between the two tensors, which are number converted from the bases. Choosing the minimum sun is as choosing the best allignment. Here rise the proble that different bases relate different numbe and therefore different penalties when mismach are found. In contranst is possible, but risky saing that since long reads will be preferably used, this consequence will be dumpened from the length of the reads.

Ireally need to keep all of them in a tensor or maybe I only need the min value and the index of it???

In [79]:
seq1 = "ATCATATGCA"
seq2 = "ATC"

tf1 = tf.constant([ord(c) for c in seq1])
tf2 = tf.constant([ord(c) for c in seq2])

min_shift_tensor = min(tf1.shape[0], tf2.shape[0])
max_shape_tensor = max(tf1.shape[0], tf2.shape[0])
num_iterations = (max_shape_tensor + min_shift_tensor - 1) / 2
int_num_iteration = (max_shape_tensor + min_shift_tensor - 1) // 2

ic(num_iterations)
ic(int_num_iteration)

alone = False # This is for the 4.5 iteration

if int_num_iteration < num_iterations:
    alone = True

print(f"Max lenght of the two tensors : {max_shape_tensor}, the min: {min_shift_tensor}")
print(f"Tensor one in his completness: {tf1}")
print(f"Tensor two in his completness: {tf2}\n")
ic(alone)

for i in range(int_num_iteration):
    if i < min_shift_tensor:
        print(f"Iteration : {i}")
        print(f"Tensor one : {tf1[:(i+1)]}")
        print(f"Tensor two : {tf2[-(i+1):]}\nThe opposite frame work")
        print(f"Tensor one : {tf1[-(i+1):]}")
        print(f"Tensor two : {tf2[:(i+1)]}\n")
    if i >= min_shift_tensor:
        print(f"Iteration : {i}")
        print(f"Tensor one : {tf1[i-min_shift_tensor+1:(i+1)]}")
        print(f"Tensor two : {tf2[-(i+1):]}\nThe opposite frame work")
        print(f"Tensor one : {tf1[-(i+1):-(i-min_shift_tensor+1)]}")
        print(f"Tensor two : {tf2[:(i+1)]}\n")
    if (i == int_num_iteration-1) and alone:
        i += 1
        print("Extra single iteration")
        print(f"Iteration : {i}")
        print(f"Tensor one : {tf1[i-min_shift_tensor+1:(i+1)]}")
        print(f"Tensor two : {tf2[-(i+1):]}")

ic| num_iterations: 6.0
ic| int_num_iteration: 6
ic| alone: False


Max lenght of the two tensors : 10, the min: 3
Tensor one in his completness: [65 84 67 65 84 65 84 71 67 65]
Tensor two in his completness: [65 84 67]

Iteration : 0
Tensor one : [65]
Tensor two : [67]
The opposite frame work
Tensor one : [65]
Tensor two : [65]

Iteration : 1
Tensor one : [65 84]
Tensor two : [84 67]
The opposite frame work
Tensor one : [67 65]
Tensor two : [65 84]

Iteration : 2
Tensor one : [65 84 67]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [71 67 65]
Tensor two : [65 84 67]

Iteration : 3
Tensor one : [84 67 65]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [84 71 67]
Tensor two : [65 84 67]

Iteration : 4
Tensor one : [67 65 84]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [65 84 71]
Tensor two : [65 84 67]

Iteration : 5
Tensor one : [65 84 65]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [84 65 84]
Tensor two : [65 84 67]



In [31]:
f = np.array([1,2,3])
ic(f.shape)


reads = ["ATCCATGCT", "AGCTAGGAGATCGA", "CTAGGATGCATAGG", "TGCATAGGACATAGCTAGGAGATCGAGATCACCAGA", "ATGCATAGGACAT", "TATATGCGCTAG", "AGATATATATGCGC"]
ncpus = 2
split_1 = reads[:len(reads)//2]
split_2 = reads[len(reads)//2:]

ic(split_1)
ic(split_2)

tf_split_1 = tf.ragged.constant([[ord(c) for c in read] for read in split_1])
tf_split_2 = tf.ragged.constant([[ord(c) for c in read] for read in split_2])

# t = Thread(target=,args=)

ic| f.shape: (3,)


ic| split_1: ['ATCCATGCT', 'AGCTAGGAGATCGA', 'CTAGGATGCATAGG']
ic| split_2: ['TGCATAGGACATAGCTAGGAGATCGAGATCACCAGA',
              'ATGCATAGGACAT',
              'TATATGCGCTAG',
              'AGATATATATGCGC']
